In [1]:
import tkinter
import numpy as np
import pandas as pd
from tkinter import *
from tkinter import filedialog
from tkinter import ttk

import info_gain
import preprocess
import quality_predict
import parameter_optimize

# 读取 & 预处理数据

In [2]:
def load_data():
    def close_window():
        csv_reader.destroy()
        
    csv_reader = Tk()
    csv_reader.title("读取数据")
    
    width = 580
    height = 400
    screenwidth = csv_reader.winfo_screenwidth()  
    screenheight = csv_reader.winfo_screenheight() 
    alignstr = '%dx%d+%d+%d' % (width, height, (screenwidth-width)/2, (screenheight-height)/2)   
    csv_reader.geometry(alignstr)

    frame_data = Frame(csv_reader)
    frame_btn = Frame(csv_reader)
    
    textLabel = Label(frame_data,
                      text="零件数据",
                      justify=LEFT)
    textLabel.pack(side=LEFT)
    
    columns = ("编号", "径向切深", "轴向切深", "主轴转速", "质量标签")
    treeview = ttk.Treeview(frame_data, height=13, show="headings", columns=columns)
    treeview.column("编号", width=100, anchor='center')
    treeview.column("径向切深", width=100, anchor='center')
    treeview.column("轴向切深", width=100, anchor='center')
    treeview.column("主轴转速", width=100, anchor='center')
    treeview.column("质量标签", width=100, anchor='center')

    treeview.heading("编号", text="编号")
    treeview.heading("径向切深", text="径向切深")
    treeview.heading("轴向切深", text="轴向切深")
    treeview.heading("主轴转速", text="主轴转速")
    treeview.heading("质量标签", text="质量标签")
    treeview.pack()
    
    file_name = filedialog.askopenfilename(title="打开文件", filetypes=[("S2out", "*.csv"), ("All Files", "*")])
    raw_df = preprocess.GUI_csv_to_df(file_name)
    
    for index, row in raw_df.iterrows():
        treeview.insert('', index, values=(index, row['Ae'], row["Ap"], row["Rs"], row["label"]))
    
    btn1 = Button(frame_btn, text="打开文件", font =("宋体",20,'bold'), width=13, height=1, command=close_window)
    btn1.pack(side="left")
    
    frame_data.pack()
    frame_btn.pack()
    csv_reader.mainloop()
    return file_name

# 信息增益计算

In [3]:
def info_gain_entry():
    def close_window():
        info_gain_entry_window.destroy()
        
    info_gain_entry_window = Tk()
    info_gain_entry_window.title("信息增益计算")
    
    btn1 = Button(info_gain_entry_window, text="信息增益计算", font =("宋体",20,'bold'), width=13, height=4, command=close_window)
    btn1.pack()
    
    info_gain_entry_window.mainloop()

def info_gain_func(file_path):
    raw_df = preprocess.GUI_csv_to_df(file_path)
    columns_entropy = [(col, info_gain.calcu_each_gain(raw_df[col], raw_df)) for col in raw_df.iloc[:, :-1]]
    def close_window():
        info_gain_window.destroy()
        
    info_gain_window = Tk()
    info_gain_window.title("信息增益计算结果")
    
    width = 400
    height = 150
    screenwidth = info_gain_window.winfo_screenwidth()
    screenheight = info_gain_window.winfo_screenheight() 
    alignstr = '%dx%d+%d+%d' % (width, height, (screenwidth-width)/2, (screenheight-height)/2)
    info_gain_window.geometry(alignstr)
    
    frame_result = Frame(info_gain_window)
    frame_btn = Frame(info_gain_window)
    
    textLabel = Label(frame_result,
                      text="信息增益计算结果",
                      justify=LEFT)
    textLabel.pack(side=LEFT)    
    
    columns = ("工艺参数", "信息增益")
    treeview = ttk.Treeview(frame_result, height=3, show="headings", columns=columns)
    treeview.column("工艺参数", width=100, anchor='center')
    treeview.column("信息增益", width=100, anchor='center')

    treeview.heading("工艺参数", text="工艺参数")
    treeview.heading("信息增益", text="信息增益")
    treeview.pack()
    
    columns_entropy_list = []
    cell1 = []
    cell1.append("径向切深")
    cell1.append(columns_entropy[0][1])
    cell2 = []
    cell2.append("轴向切深")
    cell2.append(columns_entropy[1][1])
    cell3 = []
    cell3.append("主轴转速")
    cell3.append(columns_entropy[2][1])
    columns_entropy_list.append(cell1)
    columns_entropy_list.append(cell2)
    columns_entropy_list.append(cell3)
    
    for index, row in enumerate(columns_entropy_list):
        treeview.insert('', index, values=(row[0], row[1]))
    
#     for index, row in enumerate(columns_entropy):
#         treeview.insert('', index, values=(row[0], row[1]))
        
    btn1 = Button(frame_btn, text="下一步", font =("宋体",20,'bold'), width=8, height=1, command=close_window)
    btn1.pack(side="left")
    
    frame_result.pack()
    frame_btn.pack()
    info_gain_window.mainloop()
    return columns_entropy

# 零件质量预测

In [4]:
def quality_predict_entry():
    def close_window():
        quality_predict_entry_window.destroy()
        
    quality_predict_entry_window = Tk()
    quality_predict_entry_window.title("训练零件质量预测模型")

    btn1 = Button(quality_predict_entry_window, text="训练零件质量预测模型", font =("宋体",20,'bold'), width=13, height=4, command=close_window)
    btn1.pack()

    quality_predict_entry_window.mainloop()

def quality_predict_func(file_path):
    def close_window():
        training_loss.destroy()

    training_df = preprocess.GUI_csv_to_df(file_path)
    feature_matrix, feature_matrix_mean, feature_matrix_std = preprocess.feature_eng(training_df)
    label_matrix = preprocess.label_eng(training_df)
    generator = quality_predict.cv_generator(feature_matrix, label_matrix)
    running_model = quality_predict.model_XGBoost()
    
    training_loss = Tk()
    training_loss.title("模型准确率")

    width = 600
    height = 450
    screenwidth = training_loss.winfo_screenwidth()  
    screenheight = training_loss.winfo_screenheight() 
    alignstr = '%dx%d+%d+%d' % (width, height, (screenwidth-width)/2, (screenheight-height)/2)   
    training_loss.geometry(alignstr)

    frame_data = Frame(training_loss)
    frame_btn = Frame(training_loss)
    
    textLabel = Label(frame_data,
                      text="模型准确率",
                      justify=LEFT)
    textLabel.pack(side=LEFT)
    
    columns = ("模型准确率")
    treeview = ttk.Treeview(frame_data, height=21, show="headings", columns=columns)
    treeview.column("模型准确率", width=470, anchor='w')
    treeview.heading("模型准确率", text="模型准确率")
    treeview.pack()
    
    visual_str = quality_predict.GUI_frame_classification(generator,
                                                          running_model,
                                                          feature_matrix,
                                                          label_matrix,
                                                          "./model_output/",
                                                          treeview)
    
    btn1 = Button(frame_btn, text="下一步", font =("宋体",20,'bold'), width=13, height=1, command=close_window)
    btn1.pack(side="left")
    
    frame_data.pack()
    frame_btn.pack()
    training_loss.mainloop()

def quality_predict_func2():
    def close_window():
        quality_predict_wind1.destroy()
        
    quality_predict_wind1 = Tk()
    quality_predict_wind1.title("零件质量预测数据")
    
    width = 500
    height = 160
    screenwidth = quality_predict_wind1.winfo_screenwidth()  
    screenheight = quality_predict_wind1.winfo_screenheight() 
    alignstr = '%dx%d+%d+%d' % (width, height, (screenwidth-width)/2, (screenheight-height)/2)   
    quality_predict_wind1.geometry(alignstr)

    frame_data = Frame(quality_predict_wind1)
    frame_btn = Frame(quality_predict_wind1)
    
    textLabel = Label(frame_data,
                      text="预测零件数据",
                      justify=LEFT)
    textLabel.pack(side=LEFT)
    
    columns = ("编号", "径向切深", "轴向切深", "主轴转速", "原始质量标签")
    treeview = ttk.Treeview(frame_data, height=5, show="headings", columns=columns)
    treeview.column("编号", width=100, anchor='center')
    treeview.column("径向切深", width=100, anchor='center')
    treeview.column("轴向切深", width=100, anchor='center')
    treeview.column("主轴转速", width=100, anchor='center')
    treeview.column("原始质量标签", width=100, anchor='center')

    treeview.heading("编号", text="编号")
    treeview.heading("径向切深", text="径向切深")
    treeview.heading("轴向切深", text="轴向切深")
    treeview.heading("主轴转速", text="主轴转速")
    treeview.heading("原始质量标签", text="原始质量标签")
    treeview.pack()
    
    file_name = filedialog.askopenfilename(title="打开文件", filetypes=[("S2out", "*.csv"), ("All Files", "*")])
    raw_df = preprocess.GUI_csv_to_df(file_name)
    
    for index, row in raw_df.iterrows():
        treeview.insert('', index, values=(index, row['Ae'], row["Ap"], row["Rs"], row["label"]))
    
    btn1 = Button(frame_btn, text="下一步", font =("宋体",20,'bold'), width=13, height=1, command=close_window)
    btn1.pack(side="left")
    
    frame_data.pack()
    frame_btn.pack()
    quality_predict_wind1.mainloop()
    return file_name

def quality_predict_func3(file_path):
    def close_window():
        quality_predict_wind2.destroy()
        
    quality_predict_wind2 = Tk()
    quality_predict_wind2.title("零件质量预测结果")
    
    width = 600
    height = 160
    screenwidth = quality_predict_wind2.winfo_screenwidth()  
    screenheight = quality_predict_wind2.winfo_screenheight() 
    alignstr = '%dx%d+%d+%d' % (width, height, (screenwidth-width)/2, (screenheight-height)/2)   
    quality_predict_wind2.geometry(alignstr)

    frame_data = Frame(quality_predict_wind2)
    frame_btn = Frame(quality_predict_wind2)
    
    textLabel = Label(frame_data,
                      text="预测结果",
                      justify=LEFT)
    textLabel.pack(side=LEFT)
    
    columns = ("编号", "径向切深", "轴向切深", "主轴转速", "预测结果")
    treeview = ttk.Treeview(frame_data, height=5, show="headings", columns=columns)
    treeview.column("编号", width=100, anchor='center')
    treeview.column("径向切深", width=100, anchor='center')
    treeview.column("轴向切深", width=100, anchor='center')
    treeview.column("主轴转速", width=100, anchor='center')
    treeview.column("预测结果", width=100, anchor='center')

    treeview.heading("编号", text="编号")
    treeview.heading("径向切深", text="径向切深")
    treeview.heading("轴向切深", text="轴向切深")
    treeview.heading("主轴转速", text="主轴转速")
    treeview.heading("预测结果", text="预测结果")
    treeview.pack()
    
    feature_matrix, pre_df = quality_predict.GUI_pred_sample_reader(file_path)
    load_model_dir = "./model_output/" + "model_1"
    pred_result = quality_predict.GUI_qual_pred(load_model_dir, feature_matrix)
    pre_df['label'] = pred_result
    
    for index, row in pre_df.iterrows():
        treeview.insert('', index, values=(index, row['Ae'], row["Ap"], row["Rs"], row["label"]))
    
    btn1 = Button(frame_btn, text="下一步", font =("宋体",20,'bold'), width=13, height=1, command=close_window)
    btn1.pack(side="left")
    
    frame_data.pack()
    frame_btn.pack()
    quality_predict_wind2.mainloop()

# 工艺参数优化

In [5]:
def parameter_optimize_func():
    def close_window():
        parameter_optimize_wind.destroy()
        
    parameter_optimize_wind = Tk()
    parameter_optimize_wind.title("工艺参数优化")
    
    width = 600
    height = 150
    screenwidth = parameter_optimize_wind.winfo_screenwidth()  
    screenheight = parameter_optimize_wind.winfo_screenheight() 
    alignstr = '%dx%d+%d+%d' % (width, height, (screenwidth-width)/2, (screenheight-height)/2)   
    parameter_optimize_wind.geometry(alignstr)

    frame_data = Frame(parameter_optimize_wind)
    frame_btn = Frame(parameter_optimize_wind)
    
    textLabel = Label(frame_data,
                      text="未优化工艺参数",
                      justify=LEFT)
    textLabel.pack(side=LEFT)
    
    columns = ("编号", "径向切深", "轴向切深", "零件质量", "主轴转速")
    treeview = ttk.Treeview(frame_data, height=5, show="headings", columns=columns)
    treeview.column("编号", width=100, anchor='center')
    treeview.column("径向切深", width=100, anchor='center')
    treeview.column("轴向切深", width=100, anchor='center')
    treeview.column("零件质量", width=100, anchor='center')
    treeview.column("主轴转速", width=100, anchor='center')

    treeview.heading("编号", text="编号")
    treeview.heading("径向切深", text="径向切深")
    treeview.heading("轴向切深", text="轴向切深")
    treeview.heading("零件质量", text="零件质量")
    treeview.heading("主轴转速", text="主轴转速")
    treeview.pack()
    
    file_name = filedialog.askopenfilename(title="打开文件", filetypes=[("S2out", "*.csv"), ("All Files", "*")])
    raw_df = preprocess.GUI_csv_to_df(file_name)
    
    for index, row in raw_df.iterrows():
        treeview.insert('', index, values=(index, row["Ae"], row["Ap"], "不合格", row["Rs"]))
    
    btn1 = Button(frame_btn, text="优化工艺参数", font =("宋体",20,'bold'), width=13, height=1, command=close_window)
    btn1.pack(side="left")
    
    frame_data.pack()
    frame_btn.pack()
    parameter_optimize_wind.mainloop()
    return file_name

def parameter_optimize_func2(file_path):
    def close_window():
        parameter_optimize_wind2.destroy()
        
    parameter_optimize_wind2 = Tk()
    parameter_optimize_wind2.title("工艺参数优化结果")
    
    #### Parameter optimize model training process ####
    
    file_name = "./dataset/data_entropy.csv"
    feature_matrix, feature_mean, feature_std, feature_label = parameter_optimize.GUI_op_training_data(file_name)
    opt_feature, opt_feature_df = parameter_optimize.GUI_op_sample_reader(file_path, feature_mean, feature_std)
    opt_result = parameter_optimize.GUI_para_optimize(feature_matrix, feature_label, opt_feature)
    opt_feature_df["Rs"] = opt_result
    
    width = 600
    height = 160
    screenwidth = parameter_optimize_wind2.winfo_screenwidth()  
    screenheight = parameter_optimize_wind2.winfo_screenheight() 
    alignstr = '%dx%d+%d+%d' % (width, height, (screenwidth-width)/2, (screenheight-height)/2)   
    parameter_optimize_wind2.geometry(alignstr)

    frame_data = Frame(parameter_optimize_wind2)
    frame_btn = Frame(parameter_optimize_wind2)
    
    textLabel = Label(frame_data,
                      text="工艺参数优化结果",
                      justify=LEFT)
    textLabel.pack(side=LEFT)
    
    columns = ("编号", "径向切深", "轴向切深", "零件质量", "主轴转速")
    treeview = ttk.Treeview(frame_data, height=5, show="headings", columns=columns)
    treeview.column("编号", width=100, anchor='center')
    treeview.column("径向切深", width=100, anchor='center')
    treeview.column("轴向切深", width=100, anchor='center')
    treeview.column("零件质量", width=100, anchor='center')
    treeview.column("主轴转速", width=100, anchor='center')

    treeview.heading("编号", text="编号")
    treeview.heading("径向切深", text="径向切深")
    treeview.heading("轴向切深", text="轴向切深")
    treeview.heading("零件质量", text="零件质量")
    treeview.heading("主轴转速", text="主轴转速")
    treeview.pack()
    
    
    for index, row in opt_feature_df.iterrows():
        treeview.insert('', index, values=(index, row['Ae'], row["Ap"], "合格", row["Rs"]))
    
    btn1 = Button(frame_btn, text="下一步", font =("宋体",20,'bold'), width=13, height=1, command=close_window)
    btn1.pack(side="left")
    
    frame_data.pack()
    frame_btn.pack()
    parameter_optimize_wind2.mainloop()

In [6]:
def main():
    file_path = load_data()
    info_gain_entry()
    columns_entropy = info_gain_func(file_path)
    quality_predict_entry()
    quality_predict_func(file_path)
    pred_file_path = quality_predict_func2()
    quality_predict_func3(pred_file_path)
    opt_file_path = parameter_optimize_func()
    parameter_optimize_func2(opt_file_path)

In [7]:
main()

 *** Prediction model *** 
XGBClassifier(alpha=2, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, eta=1, gamma=0,
       learning_rate=0.1, max_delta_step=1000, max_depth=4,
       min_child_weight=5, missing=None, n_estimators=200, n_jobs=1,
       nthread=None, num_boost_round=5, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=5,
       seed=None, silent=None, subsample=1, verbosity=1)
Writing trained model into dir : ./model_output/model_1
Writing trained model into dir : ./model_output/model_2
Writing trained model into dir : ./model_output/model_3
Writing trained model into dir : ./model_output/model_4
Writing trained model into dir : ./model_output/model_5
Writing trained model into dir : ./model_output/model_6
Writing trained model into dir : ./model_output/model_7
Writing trained model into dir : ./model_output/model_8
Writing trained model into dir : ./model_output/model_9
Wri

/Users/xuejingyuan/Desktop/Github/Process-Parameter-Optimization/quality_predict.py:231: RuntimeWarning: invalid value encountered in true_divide
  pro_feature_normalized = (pro_feature - pro_feature_mean)/pro_feature_std
